<a href="https://colab.research.google.com/github/sanjogmish/Ted-Talk-Views-Prediction/blob/main/Modelling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV,cross_validate,RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder

In [131]:
!pip install category_encoders 
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [132]:
df = pd.read_csv('/content/drive/MyDrive/Capstone 3 Regression/ted_new.csv')

In [133]:
df.head(10)

,Unnamed: 0,talk_id,title,speaker_1,all_speakers,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript,occupation,no_of_lang,cleaned_description,release_day,release_month,release_year,cleaned_transcription,talk_category
0,0,1,Averting the climate crisis,Al Gore,{0: 'Al Gore'},{0: 'Nobel Laureate Al Gore focused the world’...,3523392,2006-02-25,2006-06-27,TED2006,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",272.0,16.283333,"['alternative energy', 'cars', 'climate change...","{243: 'New thinking on the climate crisis', 54...",https://www.ted.com/talks/al_gore_averting_the...,With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre...",climate advocate,43,humor humanity exude An Inconvenient Truth Al ...,Tuesday,June,2006,thank Chris truly great honor opportunity come...,Culture and society
1,1,92,The best stats you've ever seen,Hans Rosling,{0: 'Hans Rosling'},"{0: 'In Hans Rosling’s hands, data sings. Glob...",14501685,2006-02-22,2006-06-27,TED2006,en,"['ar', 'az', 'bg', 'bn', 'bs', 'cs', 'da', 'de...",628.0,19.833333,"['Africa', 'Asia', 'Google', 'demo', 'economic...","{2056: ""Own your body's data"", 2296: 'A visual...",https://www.ted.com/talks/hans_rosling_the_bes...,You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac...",global health expert; data visionary,49,datum present like drama urgency sportscaster ...,Tuesday,June,2006,year ago task teach global development swedish...,International topics
2,2,7,Simplicity sells,David Pogue,{0: 'David Pogue'},{0: 'David Pogue is the personal technology co...,1920832,2006-02-24,2006-06-27,TED2006,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",124.0,21.433333,"['computers', 'entertainment', 'interface desi...","{1725: '10 top time-saving tech tips', 2274: '...",https://www.ted.com/talks/david_pogue_simplici...,New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun...",technology columnist,26,New York Times columnist David Pogue aim techn...,Tuesday,June,2006,music sound silence Simon Garfunkel hello voic...,Culture and society
3,3,53,Greening the ghetto,Majora Carter,{0: 'Majora Carter'},{0: 'Majora Carter redefined the field of envi...,2664069,2006-02-26,2006-06-27,TED2006,en,"['ar', 'bg', 'bn', 'ca', 'cs', 'de', 'en', 'es...",219.0,18.600000,"['MacArthur grant', 'activism', 'business', 'c...",{1041: '3 stories of local eco-entrepreneurshi...,https://www.ted.com/talks/majora_carter_greeni...,"In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...,activist for environmental justice,35,emotionally charge talk MacArthur win activist...,Tuesday,June,2006,today happy hear sustainable development save ...,Climate change
4,4,66,Do schools kill creativity?,Sir Ken Robinson,{0: 'Sir Ken Robinson'},"{0: ""Creativity expert Sir Ken Robinson challe...",65051954,2006-02-25,2006-06-27,TED2006,en,"['af', 'ar', 'az', 'be', 'bg', 'bn', 'ca', 'cs...",4931.0,19.400000,"['children', 'creativity', 'culture', 'dance',...","{865: 'Bring on the learning revolution!', 173...",https://www.ted.com/talks/sir_ken_robinson_do_...,Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...,author,62,Sir Ken Robinson entertaining profoundly case ...,Tuesday,June,2006,good morning audience good great blow away thi...,Finance and economics
5,5,49,Behind the design of Seattle's library,Joshua Prince-Ramus,{0: 'Joshua Prince-Ramus'},{0: 'Joshua Prince-Ramus is best known as arch...,1208138,2006-02-23,2006-07-10,TED2006,en,"['ar', 'bg', 'en', 'es', 'fr', 'he', 'it', 'ja...",48.0,19.966667,"['architecture', 'collaboration', 'culture', '...",{750: 'Building a theater that remakes itself'...,https://www.ted.com/talks/joshua_prince_ramus_...,Architect Joshua Prince-Ramus takes the audien...,I'm goin

In [134]:
df.drop(['Unnamed: 0','talk_id','title','speaker_1','all_speakers','about_speakers','recorded_date','published_date','event','native_lang','available_lang','topics',
          'related_talks','url','description','transcript','occupation','cleaned_description','release_day','release_month','release_year','cleaned_transcription'],axis=1,inplace=True)

In [135]:
df.head()

,views,comments,duration,no_of_lang,talk_category
0,3523392,272.0,16.283333,43,Culture and society
1,14501685,628.0,19.833333,49,International topics
2,1920832,124.0,21.433333,26,Culture and society
3,2664069,219.0,18.600000,35,Climate change
4,65051954,4931.0,19.400000,62,Finance and economics


In [136]:
df.describe()

,views,comments,duration,no_of_lang
count,4.005000e+03,3350.000000,4005.000000,4005.000000
mean,2.148006e+06,161.997015,12.066854,25.304120
std,3.451226e+06,268.838922,6.029591,9.038567
min,0.000000e+00,0.000000,1.000000,1.000000
25%,8.820690e+05,38.000000,6.550000,20.000000
50%,1.375508e+06,89.000000,12.300000,25.000000
75%,2.133110e+06,188.000000,16.233333,31.000000
max,6.505195e+07,6449.000000,65.366667,73.000000


In [137]:
# Removing null values in 'comments' with median

df['comments'] = np.where(df['comments'].isnull(),89,df['comments'])

In [138]:
# Outliers Removal

# Comments Column

Q1 = np.percentile(df['comments'],25)
Q3 = np.percentile(df['comments'],75)
IQR = Q3 - Q1
median_comments = df.loc[((Q1 - 1.5* IQR) <= df['comments']) & (df['comments'] <= (Q3 + 1.5* IQR)),'comments'].median()
df['comments'] = np.where(((Q1 - 1.5* IQR) > df['comments']) & (df['comments']> (Q3 + 1.5* IQR)), median_comments,df['comments'])

In [139]:
# Duration Column

Q1 = np.percentile(df['duration'],25)
Q3 = np.percentile(df['duration'],75)
IQR = Q3 - Q1
median_duration = df.loc[((Q1 - 1.5* IQR) <= df['duration']) & (df['duration'] <= (Q3 + 1.5* IQR)),'duration'].median()
df['duration'] = np.where(((Q1 - 1.5* IQR) > df['duration']) & (df['duration']> (Q3 + 1.5* IQR)), median_duration,df['duration'])

In [140]:
# Number of languages

Q1 = np.percentile(df['no_of_lang'],25)
Q3 = np.percentile(df['no_of_lang'],75)
IQR = Q3 - Q1
median_no_of_lang = df.loc[((Q1 - 1.5* IQR) <= df['no_of_lang']) & (df['no_of_lang'] <= (Q3 + 1.5* IQR)),'no_of_lang'].median()
df['no_of_lang'] = np.where(((Q1 - 1.5* IQR) > df['no_of_lang']) & (df['no_of_lang']> (Q3 + 1.5* IQR)), median_no_of_lang,df['no_of_lang'])

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   views          4005 non-null   int64  
 1   comments       4005 non-null   float64
 2   duration       4005 non-null   float64
 3   no_of_lang     4005 non-null   float64
 4   talk_category  4005 non-null   object 
dtypes: float64(3), int64(1), object(1)
memory usage: 156.6+ KB


## **Splitting of dataset into train and test**

In [142]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [144]:
# Target encoding

encoder=ce.TargetEncoder(cols=['talk_category'])
encoded_values_array = encoder.fit_transform(X_train[['talk_category']],y_train).values

In [145]:
encoded_values_array

array([[1401646.62676056],
       [1562257.97307692],
       [1562257.97307692],
       ...,
       [3027476.34184397],
       [3027476.34184397],
       [1562257.97307692]])

In [146]:
X_train['talk_category']  = encoded_values_array

In [147]:
X_train

,comments,duration,no_of_lang,talk_category
3180,89.0,4.550000,21.0,1.401647e+06
2513,15.0,14.900000,16.0,1.562258e+06
3696,89.0,14.416667,4.0,1.562258e+06
3001,74.0,12.900000,17.0,1.746913e+06
3300,89.0,5.583333,31.0,3.027476e+06
...,...,...,...,...
3839,13.0,6.566667,23.0,2.327546e+06
1096,45.0,16.766667,25.0,1.401647e+06
3980,9.0,10.833333,7.0,3.027476e+06
235,88.0,17.300000,24.0,3.027476e+06


In [148]:
# Encoding test data

encoded_values_array_test = encoder.transform(X_test[['talk_category']],y_test).values
X_test['talk_category']  = encoded_values_array_test

In [151]:
# Scaling the data

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [153]:
X_test = scaler.transform(X_test)

**LINEAR REGRESSION**

In [155]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression()

In [156]:
y_train_pred = regressor.predict(X_train)
y_pred = regressor.predict(X_test)

In [157]:
r2_score(y_train,y_train_pred)

0.22297151290020922

In [158]:
r2_score(y_test,y_pred)

0.37268089237052715

In [172]:
# TUNING

alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)

lr_grid = GridSearchCV(estimator=Ridge(),
                       param_grid = param_grid,refit=True,n_jobs=-1,
                       cv = 5, verbose=2, scoring='r2')

grid_result = lr_grid.fit(X_train,y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


In [174]:
grid_result.best_score_

0.2044323072637194

METHOD 2

In [244]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=8)

In [245]:
X_train2.values[:,0:3]

array([[111.0, 15.466666666666667, 24.0],
       [65.0, 16.416666666666668, 20.0],
       [89.0, 4.516666666666667, 21.0],
       ...,
       [53.0, 15.5, 21.0],
       [178.0, 8.8, 33.0],
       [324.0, 18.633333333333333, 22.0]], dtype=object)

In [246]:
# one hot encoding

encoder=OneHotEncoder(sparse=False)
encoded_values_array = encoder.fit_transform(X_train2['talk_category'].values.reshape(-1,1))

In [247]:
X_train2.values[:,0:3]

array([[111.0, 15.466666666666667, 24.0],
       [65.0, 16.416666666666668, 20.0],
       [89.0, 4.516666666666667, 21.0],
       ...,
       [53.0, 15.5, 21.0],
       [178.0, 8.8, 33.0],
       [324.0, 18.633333333333333, 22.0]], dtype=object)

In [248]:
X_train2_array_encoded = np.concatenate((X_train2.values[:,0:3] , encoded_values_array),axis=1)

In [249]:
encoded_values_array_test = encoder.transform(X_test2['talk_category'].values.reshape(-1,1))
X_test2_array_encoded  = np.concatenate((X_test2.values[:,0:3] , encoded_values_array_test),axis=1)

In [251]:
scaler2 = StandardScaler()
X_train2_array_encoded = scaler.fit_transform(X_train2_array_encoded)
X_test2_array_encoded = scaler.transform(X_test2_array_encoded)

In [252]:
regressor = LinearRegression()
regressor.fit(X_train2_array_encoded ,y_train2)

y_train_pred = regressor.predict(X_train2_array_encoded)
y_pred = regressor.predict(X_test2_array_encoded)

In [253]:
r2_score(y_train,y_train_pred)

-0.2719794524557939

In [255]:
# random forest on target encoded values

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier() 
grid_values = {'n_estimators':[100, 110,  120], 'max_depth':[5, 6, 7]}
classifier = GridSearchCV(classifier, param_grid = grid_values, scoring = 'r2', cv=5)

# Fit the object to train dataset
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 6, 7],
                         'n_estimators': [100, 110, 120]},
             scoring='r2')

In [257]:
y_train_pred = classifier.predict(X_train)
y_pred = classifier.predict(X_test)

In [258]:
r2_score(y_train,y_train_pred)

0.39555789563269983

In [259]:
r2_score(y_test,y_pred)

0.05614147415633608